In [ ]:
import numpy as np

In [ ]:
def onehot_enc(lbl, min_val=0):
    mi= min(lbl)
    enc=np.full((len(lbl), max(lbl)-mi+1), min_val, np.int8)

    for i, x in enumerate(lbl):
        enc[i,x-mi]=1

    return enc
def onehot_dec(enc, mi=0):
    return [np.argmax(e) + mi for e in enc]

In [ ]:
labels = 0,1
enc = onehot_enc(labels)
dec = onehot_dec(enc, min(labels))

print(enc)
print(dec)

[[1 0]
 [0 1]]
[0, 1]


In [ ]:
#fungsi aktivasi sigmoid (menuju hidden layer)
#ada banyak aktivasi, tp sigmoid cocok untuk binary class
def sig(X): #feed forward -> input ke hidden layer
    return [1/(1+np.exp(-x)) for x in X]

def sigd(X): #diferensial sigmoid #backward -> output ke hidden layer
    output= []

    for i, x in enumerate(X):
        s = sig([x])[0]

        output.append(s * (1- s))

    return output

In [ ]:
#algoritma Backpropagation Neural Network untuk Training Data
def bp_fit(X, target, layer_conf, max_epoch, max_error=.1, learn_rate=.1, print_per_epoch=100):
    nin = [np.empty(1) for i in layer_conf]
    n = [np.empty(j+1) if i< len(layer_conf)-1 else np.empty(j) for i, j in enumerate(layer_conf)]
    w = np.array([np.random.rand(layer_conf[i] + 1, layer_conf[i+1]) for i in range(len(layer_conf)-1)]) #inisialisasi Bobot
    dw = [np.empty((layer_conf[i]+ 1, layer_conf[i + 1]))for i in range(len(layer_conf) - 1)]
    d = [np.empty(s) for s in layer_conf[1:]]
    din = [np.empty(s) for s in layer_conf[1:-1]]
    epoch = 0
    mse = 1

    for i in range(0, len(n)-1):
        n[i][-1]=1
    
    while (max_epoch == -1 or epoch < max_epoch) and mse > max_error:
        epoch += 1
        mse = 0

        for r in range(len(X)):
            n[0][:-1] = X[r]

            for L in range(1, len(layer_conf)):
                nin[L] = np.dot(n[L-1], w[L-1])
                n[L][:len(nin[L])] = sig(nin[L])

            e = target[r] - n[-1]
            mse += sum(e ** 2)
            d[-1] = e * sigd(nin[-1])
            dw[-1] = learn_rate * d[-1] * n[-2].reshape((-1,1))

            for L in range(len(layer_conf) -1, 1, -1):
                din[L-2] = np.dot(d[L-1], np.transpose(w[L-1][:-1]))
                d[L-2] = din[L-2] * np.array(sigd(nin[L-1]))
                dw[L-2] = (learn_rate * d[L-2]) * n[L-2].reshape((-1,1))

            w += dw

        mse /= len(X)

        if print_per_epoch >-1 and epoch % print_per_epoch == 0:
            print(f'Epoch {epoch}, MSE: {mse}')

    return w,epoch, mse

In [ ]:
#Pengujian algoritma klasifikasi Backpropagation Neural Network untuk Testing Data
def bp_predict(X, w) :
    n = [np.empty(len(i)) for i in w]
    nin = [np.empty(len(i[0])) for i in w]
    predict = []

    n.append(np.empty(len(w[-1][0])))

    for x in X:
        n[0][:-1]= x

        for L in range(0, len(w)):
            nin[L] = np.dot(n[L], w[L])
            n[L + 1][:len(nin[L])] = sig(nin[L])

        predict.append(n[-1].copy())

    return predict

In [ ]:
#mengolah data latih yang telah di preprocessing
from sklearn.metrics import accuracy_score

import pandas as pd
df=pd.read_excel("ekstraksilatih.xlsx")
df2=pd.read_excel("ekstraksiuji.xlsx")

In [ ]:
df

,fitur1,fitur2,fitur3,fitur4,fitur5,fitur6,fitur7,fitur8,fitur9,fitur10,...,fitur32,fitur33,fitur34,fitur35,fitur36,fitur37,fitur38,fitur39,fitur40,klasifikasi
0,-2.007642,2.556125,0.072266,-0.152799,-0.403144,-0.249247,-0.055301,0.055129,-0.046273,-0.090856,...,-0.090856,-0.046273,0.055129,-0.055301,-0.249247,-0.403144,-0.152799,0.072266,2.556125,0
1,-1.901769,2.613287,0.152605,-0.147798,-0.366786,-0.228099,-0.149943,0.048139,0.019961,-0.142346,...,-0.142346,0.019961,0.048139,-0.149943,-0.228099,-0.366786,-0.147798,0.152605,2.613287,0
2,-2.043165,2.524892,0.126098,-0.140519,-0.359082,-0.318049,-0.060746,0.071458,-0.061505,-0.092370,...,-0.092370,-0.061505,0.071458,-0.060746,-0.318049,-0.359082,-0.140519,0.126098,2.524892,0
3,-2.151359,2.434558,0.214443,-0.129482,-0.373807,-0.274897,-0.091357,0.051193,-0.000570,-0.105447,...,-0.105447,-0.000570,0.051193,-0.091357,-0.274897,-0.373807,-0.129482,0.214443,2.434558,0
4,-2.122244,2.469517,0.147141,-0.122743,-0.353407,-0.161182,-0.126801,0.018733,-0.009604,-0.091252,...,-0.091252,-0.009604,0.018733,-0.126801,-0.161182,-0.353407,-0.122743,0.147141,2.469517,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,-1.506781,2.565440,0.186189,0.863261,-0.436089,0.319979,-0.470863,0.082793,-0.128820,-0.112701,...,-0.112701,-0.128820,0.082793,-0.470863,0.319979,-0.436089,0.863261,0.186189,2.565440,0
1436,-1.530720,2.510072,-0.096379,0.883384,-0.662597,-0.087853,-0.377537,0.442490,-0.171413,-0.153466,...,-0.153466,-0.171413,0.442490,-0.377537,-0.087853,-0.662597,0.883384,-0.096379,2.510072,0
1437,-1.826050,2.345515,0.221958,0.932037,-0.710399,0.123859,-0.508489,0.196079,-0.026493,-0.128546,...,-0.128546,-0.026493,0.196079,-0.508489,0.123859,-0.710399,0.932037,0.221958,2.345515,0
1438,-1.544674,2.496480,0.081013,0.996558,-0.234049,0.273444,-0.347068,0.125960,-0.068548,-0.060886,...,-0.060886,-0.068548,0.125960,-0.347068,0.273444,-0.234049,0.996558,0.081013,2.496480,1


In [ ]:
df2

,fitur1,fitur2,fitur3,fitur4,fitur5,fitur6,fitur7,fitur8,fitur9,fitur10,...,fitur32,fitur33,fitur34,fitur35,fitur36,fitur37,fitur38,fitur39,fitur40,klasifikasi
0,-1.563190,2.708097,0.284603,0.116453,-0.291089,-0.117732,-0.298117,0.103194,0.056427,0.072161,...,0.072161,0.056427,0.103194,-0.298117,-0.117732,-0.291089,0.116453,0.284603,2.708097,0
1,-1.807739,2.594554,0.451813,0.118015,-0.370432,-0.169615,-0.362202,-0.005054,0.063080,0.329760,...,0.329760,0.063080,-0.005054,-0.362202,-0.169615,-0.370432,0.118015,0.451813,2.594554,0
2,-1.732912,2.680671,0.306235,0.195759,-0.110984,-0.160148,-0.220760,0.129448,-0.088793,0.103150,...,0.103150,-0.088793,0.129448,-0.220760,-0.160148,-0.110984,0.195759,0.306235,2.680671,0
3,-1.814899,2.518094,0.730282,0.226388,-0.281736,-0.197983,-0.380463,0.006385,0.057717,0.301684,...,0.301684,0.057717,0.006385,-0.380463,-0.197983,-0.281736,0.226388,0.730282,2.518094,0
4,-1.791928,2.547804,0.530301,0.228701,-0.558956,-0.091406,-0.210834,-0.016335,0.069991,0.156068,...,0.156068,0.069991,-0.016335,-0.210834,-0.091406,-0.558956,0.228701,0.530301,2.547804,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,-1.885853,2.422644,0.335060,0.873722,-0.016561,-0.168762,-0.273735,0.092996,0.014971,0.149217,...,0.149217,0.014971,0.092996,-0.273735,-0.168762,-0.016561,0.873722,0.335060,2.422644,0
356,-1.963663,2.362706,0.510250,0.875082,-0.117414,-0.108921,-0.134167,0.040691,-0.151352,0.089971,...,0.089971,-0.151352,0.040691,-0.134167,-0.108921,-0.117414,0.875082,0.510250,2.362706,0
357,-1.990777,2.324929,0.328662,0.876093,-0.080270,-0.082970,-0.299518,0.161501,0.061864,0.110539,...,0.110539,0.061864,0.161501,-0.299518,-0.082970,-0.080270,0.876093,0.328662,2.324929,1
358,-1.673359,2.457289,0.417346,0.902758,-0.003065,0.095007,-0.439994,0.147094,0.103732,0.164548,...,0.164548,0.103732,0.147094,-0.439994,0.095007,-0.003065,0.902758,0.417346,2.457289,1


In [ ]:
features=df.drop(["klasifikasi"],axis=1)
X_test = df2.drop(["klasifikasi"],axis=1)
features.shape

(1440, 40)

In [ ]:
X_test.shape

(360, 40)

In [ ]:
indextable = []
for i in range(40):
    indextable.append("fitur" + str(i+1))
labels=df.drop(indextable, axis=1)
kelas=labels.to_numpy()
ytestemp= df2.drop(indextable, axis=1)
ytestemp2 = ytestemp.to_numpy()

In [ ]:
def featuresarray(nparray):
    result = np.empty(len(nparray), int)
    for i in range(len(nparray)):
        result[i] = nparray[i,0]
    return result

In [ ]:
out = featuresarray(kelas)

In [ ]:
# data latih
X = features.to_numpy()
Y = onehot_enc(out)
# data uji
X_test2 = X_test.to_numpy()
Y_test = featuresarray(ytestemp2)

In [ ]:
w, ep, mse = bp_fit(X, Y, layer_conf=(40, 37, 2), learn_rate=.1, max_epoch=3000, max_error=.1,print_per_epoch=50)

C:\Users\NIKONU~1\AppData\Local\Temp/ipykernel_11288/2630296327.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  w = np.array([np.random.rand(layer_conf[i] + 1, layer_conf[i+1]) for i in range(len(layer_conf)-1)]) #inisialisasi Bobot
C:\Users\NIKONU~1\AppData\Local\Temp/ipykernel_11288/2630296327.py:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  w += dw


Epoch 50, MSE: 0.18578186125859486
Epoch 100, MSE: 0.138187379009735
Epoch 150, MSE: 0.11641661183942603
Epoch 200, MSE: 0.10288223619428569


In [ ]:
print(f'Epochs: {ep}, MSE: {mse}')

Epochs: 217, MSE: 0.09985274432121523


In [ ]:
Y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0,

In [ ]:
predict = bp_predict(X_test2, w)
predict = onehot_dec(predict)
acc = accuracy_score(predict, Y_test)

In [ ]:
print(f'Output: {predict}')
print(f'TRUE  : {Y_test}')
print(f'Accuracy: {acc}')

Output: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1

In [ ]:
print("==============PENGUJIAN SUARA=============")
print("Berikut data suara untuk diuji : ")
df2=pd.read_excel("ujiML_2.xlsx")
print(df2)

==============PENGUJIAN SUARA=============
Berikut data suara untuk diuji : 
     fitur1   fitur2    fitur3    fitur4    fitur5    fitur6    fitur7  \
0 -1.544674  2.49648  0.081013  0.996558 -0.234049  0.273444 -0.347068   

    fitur8    fitur9   fitur10  ...   fitur32   fitur33  fitur34   fitur35  \
0  0.12596 -0.068548 -0.060886  ... -0.060886 -0.068548  0.12596 -0.347068   

    fitur36   fitur37   fitur38   fitur39  fitur40  klasifikasi  
0  0.273444 -0.234049  0.996558  0.081013  2.49648            1  

[1 rows x 41 columns]


In [ ]:
features2=df2.drop(["klasifikasi"],axis=1)
a = features2.to_numpy()
# alfa = np.asarray(alfa)
a
# features2.to_numpy()

array([[-1.54467362,  2.49647991,  0.08101268,  0.99655839, -0.23404853,
         0.27344357, -0.34706842,  0.12595953, -0.06854788, -0.06088645,
        -0.01583168,  0.0292231 ,  0.03688453, -0.15762288,  0.31540506,
        -0.30510692,  0.20238518, -1.02822174, -0.11267603, -2.52814326,
         2.14627733, -2.52814326, -0.11267603, -1.02822174,  0.20238518,
        -0.30510692,  0.31540506, -0.15762288,  0.03688453,  0.0292231 ,
        -0.01583168, -0.06088645, -0.06854788,  0.12595953, -0.34706842,
         0.27344357, -0.23404853,  0.99655839,  0.08101268,  2.49647991]])

In [ ]:
# alfa = features2 #var penampung nilai data latih
# beta = Y_test[2] #var penampung nilai output yang diprediksi

tes = np.empty((1,40))
tes[0,:] = a
# alfa.shape()
# features2.shape()
print("input yang diterima sama bp_predict:", tes)

input yang diterima sama bp_predict: [[-1.54467362  2.49647991  0.08101268  0.99655839 -0.23404853  0.27344357
  -0.34706842  0.12595953 -0.06854788 -0.06088645 -0.01583168  0.0292231
   0.03688453 -0.15762288  0.31540506 -0.30510692  0.20238518 -1.02822174
  -0.11267603 -2.52814326  2.14627733 -2.52814326 -0.11267603 -1.02822174
   0.20238518 -0.30510692  0.31540506 -0.15762288  0.03688453  0.0292231
  -0.01583168 -0.06088645 -0.06854788  0.12595953 -0.34706842  0.27344357
  -0.23404853  0.99655839  0.08101268  2.49647991]]


In [ ]:
kelasapa = bp_predict(tes, w)
kelasapa = onehot_dec(kelasapa)
print( "Output Prediksi : ")
print("",kelasapa)
if(kelasapa==[0]):
    print("LOCK")
elif(kelasapa==[1]):
    print("UNLOCK")

Output Prediksi : 
 [1]
UNLOCK
